# Linguistic Analysis: Text Length, ADJ Percentage

In [ ]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import pandas as pd
import pickle
from statsmodels.stats import weightstats as stests
from collections import Counter, defaultdict
import networkx as nx 
from sklearn import metrics
import statsmodels.api as sm
from main_analyze import parse_InvNameList, assign_income_rank
from interdisciplinary import compare_x1_x2

In [ ]:
import seaborn as sns 
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set(font_scale=1.4)  
df_abstract = pd.read_pickle("notebook_plots/linguistic_spacy_{}.pkl".format('abstract'))
df_abstract_Marketing = pd.read_pickle("notebook_plots/linguistic_spacy_{}.pkl".format('abstract_Marketing'))
author_abstract_len, marketing_abstract_len, year_str_list = [], [], []
year_stratify_scheme = [
    (1980, 1990),
    (1990, 2000),
    (2000, 2005),
    (2005, 2010),
    (2010, 2015),
    (2015, 2020),
]
for year_start, year_end in year_stratify_scheme:
    author_abstract_len.append(
        df_abstract.query(" `Disclosure Date`>='{}-01-01'  and `Disclosure Date`<'{}-01-01' ".format(year_start, year_end))['Title_len'].mean()
    )
    marketing_abstract_len.append(
        df_abstract_Marketing.query(" `Disclosure Date`>='{}-01-01'  and `Disclosure Date`<'{}-01-01' ".format(year_start, year_end))['Title_len'].mean()
    )
    year_str_list.append( str(year_start) + '-' + str(year_end) )
df_tmp = {
    'year': year_str_list,
    'author abstract': author_abstract_len, 
    'marketing abstract' : marketing_abstract_len,
}
df_tmp = pd.DataFrame(df_tmp)
print('df_tmp', df_tmp)
df_tmp = pd.melt(df_tmp, id_vars="year", var_name="", value_name="Length: word count")
sns.catplot(data=df_tmp, x="year", y="Length: word count", hue="", kind='bar', legend_out=False)
_ = plt.xticks(rotation=30)
print('Longer Text over time -- both abstracts')

In [ ]:
import seaborn as sns 
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set(font_scale=1.4)  
df_title= pd.read_pickle("notebook_plots/linguistic_spacy_{}.pkl".format('Title'))
title_adj_percentage_array = []
year_str_list = []
year_stratify_scheme = [
    (1980, 1990),
    (1990, 2000),
    (2000, 2005),
    (2005, 2010),
    (2010, 2015),
    (2015, 2020),
]
for year_start, year_end in year_stratify_scheme:
    title_adj_percentage_array.append(
        df_title.query(" `Disclosure Date`>='{}-01-01'  and `Disclosure Date`<'{}-01-01' ".format(year_start, year_end))['ADJ_percentage'].mean()
    )
    year_str_list.append( str(year_start) + '-' + str(year_end) )
df_tmp = {
    'year': year_str_list,
    'title': title_adj_percentage_array, 
}
df_tmp = pd.DataFrame(df_tmp)
print('df_tmp\n', df_tmp)
df_tmp = pd.melt(df_tmp, id_vars="year", var_name="", value_name="Fraction of adjectives")
sns.catplot(data=df_tmp, x="year", y="Fraction of adjectives", hue="", kind='bar', legend_out=False)
_ = plt.xticks(rotation=30)
print('Longer Text over time -- both abstracts')

In [ ]:
# FIELD = 'Title'
# FIELD = 'abstract_Marketing'
FIELD = 'abstract' 
# cached_var_name = 'Title_len' 
cached_var_name = 'ADJ_percentage' 
cache_df_path = "notebook_plots/linguistic_spacy_{}.pkl".format(FIELD)
df_all = pd.read_pickle(cache_df_path)
print('File loaded from ' + cache_df_path)
from main_analyze import label_vocab
interested_label_vocab = label_vocab[:6] 
X = df_all[['year'] + interested_label_vocab] 
Y = df_all[cached_var_name] 
X = sm.add_constant(X) 
model = sm.OLS(Y, X.astype(float)).fit()
predictions = model.predict(X) 
print_model = model.summary()
print(print_model)
print('p-values:', model.pvalues)

## Relationship with net income 

In [ ]:
# FIELD = 'Title'
FIELD = 'abstract_Marketing'
# FIELD = 'abstract' 
cache_df_path = "notebook_plots/linguistic_spacy_{}.pkl".format(FIELD)
df_all = pd.read_pickle(cache_df_path)
print('File loaded from ' + cache_df_path)
from main_analyze import label_vocab
interested_label_vocab = label_vocab[:6] 
X = df_all[['year', 'ADJ_percentage', 'Title_len'] + interested_label_vocab] 
Y = df_all['income_rank'] 
X = sm.add_constant(X) 
model = sm.OLS(Y, X.astype(float)).fit()
predictions = model.predict(X) 
print_model = model.summary()
print(print_model)
print('p-values:', model.pvalues)
